In [32]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import joblib

In [37]:
# Define the HOA algorithm with constraints on hyperparameters
def hoa_algorithm(data, population_size=10, max_iter=100, search_space=(-1, 1)):
    # Initialize population with constrained ranges
    population = np.random.uniform(search_space[0], search_space[1], (population_size, 3))  # Hyperparameters: LSTM units, learning rate, batch size
    
    # Enforce constraints
    population[:, 0] = np.clip(population[:, 0], 10, 200)  # LSTM units between 10 and 200
    population[:, 2] = np.clip(population[:, 2], 16, 64)   # Batch size between 16 and 64
    population[:, 1] = np.clip(population[:, 1], 0.0001, 0.01)  # Learning rate between 0.0001 and 0.01
    
    best_solution = population[0]
    best_score = objective_function(best_solution, data)
    
    for iteration in range(max_iter):
        for i in range(population_size):
            # Small perturbations for hyperparameters
            candidate = population[i] + np.random.uniform(-0.1, 0.1, 3)
            
            # Apply constraints to candidate solution
            candidate[0] = np.clip(candidate[0], 10, 200)  # LSTM units
            candidate[2] = np.clip(candidate[2], 16, 64)   # Batch size
            candidate[1] = np.clip(candidate[1], 0.0001, 0.01)  # Learning rate
            
            candidate_score = objective_function(candidate, data)
            
            if candidate_score < best_score:
                best_solution = candidate
                best_score = candidate_score
        
        print(f"Iteration {iteration+1}/{max_iter} - Best Score: {best_score}")
    
    return best_solution, best_score


In [34]:
# Objective function for optimization (Mean Squared Error)
def objective_function(hyperparameters, data):
    lstm_units = int(hyperparameters[0])  # Number of LSTM units
    learning_rate = hyperparameters[1]    # Learning rate
    batch_size = int(hyperparameters[2])  # Batch size

    # Data preprocessing
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[['Adj Close']].values)

    # Prepare the data for LSTM
    def create_dataset(data, time_step=60):
        X, y = [], []
        for i in range(time_step, len(data)):
            X.append(data[i-time_step:i, 0])  # past 'time_step' days
            y.append(data[i, 0])  # next day price
        return np.array(X), np.array(y)

    X, y = create_dataset(scaled_data)

    # Reshape data into 3D input for LSTM
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # Build and compile the LSTM model
    model = Sequential()
    model.add(LSTM(units=lstm_units, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(units=lstm_units, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Train the model
    model.fit(X, y, epochs=10, batch_size=batch_size, verbose=0)

    # Evaluate the model
    mse = model.evaluate(X, y, verbose=0)
    return mse

In [35]:
# Download data for training
currency_data = yf.download('USDINR=X', start='2020-01-01', end='2023-01-01')
currency_data = currency_data[['Adj Close']].dropna()

[*********************100%***********************]  1 of 1 completed


In [38]:
# Run HOA to optimize hyperparameters
best_hyperparameters, best_mse = hoa_algorithm(currency_data)
print(f"Best Hyperparameters: {best_hyperparameters}, with MSE: {best_mse}")

C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\laye

Iteration 1/100 - Best Score: 0.0009150634286925197


C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\laye

Iteration 2/100 - Best Score: 0.0009150634286925197


C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\laye

...

Iteration 98/100 - Best Score: 0.0007608685991726816


C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\laye

Iteration 99/100 - Best Score: 0.0007608685991726816


C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\laye

Iteration 100/100 - Best Score: 0.0007608685991726816
Best Hyperparameters: [1.00000000e+01 1.00000000e-02 1.60892208e+01], with MSE: 0.0007608685991726816


In [39]:
# Train the final model using the optimized hyperparameters
lstm_units = int(best_hyperparameters[0])
learning_rate = best_hyperparameters[1]
batch_size = int(best_hyperparameters[2])

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(currency_data[['Adj Close']].values)

In [40]:
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])  # past 'time_step' days
        y.append(data[i, 0])  # next day price
    return np.array(X), np.array(y)

X, y = create_dataset(scaled_data)

X = X.reshape(X.shape[0], X.shape[1], 1)

In [41]:
# Build the optimized LSTM model
model = Sequential()
model.add(LSTM(units=lstm_units, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=lstm_units, return_sequences=False))
model.add(Dense(units=1))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mean_squared_error')

C:\Users\ar415\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [42]:
# Train the model
model.fit(X, y, epochs=10, batch_size=batch_size)

Epoch 1/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - loss: 0.0491
Epoch 2/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0022
Epoch 3/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0017
Epoch 4/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0013
Epoch 5/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.0013
Epoch 6/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - loss: 0.0014
Epoch 7/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.0013
Epoch 8/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0014
Epoch 9/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0018
Epoch 10/10
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.0014


In [43]:
# Save the trained model
model.save("optimized_lstm_currency_model.h5")
print("Model saved as 'optimized_lstm_currency_model.h5'")

Model saved as 'optimized_lstm_currency_model.h5'
